In [1026]:
import numpy as np
import random as rd
from math import ceil

from collections import defaultdict

import pygame

seed = 42
rd.seed(seed)

### Definindo constantes

In [1027]:
N_H = 50
N_O = ceil(N_H/2)

In [1028]:
SCALE = 7.2

In [1029]:
BOX_SIZE = [150,100]

In [1030]:
TEMP = 1
MAX_SPEED, MIN_SPEED = -20,20

TIME_STEP = 0.01

In [1031]:
BOX_SIZE = np.array(BOX_SIZE) * SCALE
MAX_SPEED, MIN_SPEED = MAX_SPEED*SCALE, MIN_SPEED*SCALE

In [1032]:
H_INFOS = {
    'radius': 1*SCALE,
    'mass': 1
}

O_INFOS = {
    'radius': 2*SCALE,
    'mass': 16
}

### Definindo a classe de partículas

In [1033]:
class Atom:
    def __init__(self, pos, radius, mass, type):
        self.type = type

        self.pos = np.array(pos)
        self.radius = radius
        self.mass = mass
        self.speed = np.array([rd.uniform(MIN_SPEED, MAX_SPEED), rd.uniform(MIN_SPEED, MAX_SPEED)]) * TEMP
        self.accel = 0

        self.momentum = mass * self.speed
    
    def move(self):
        self.speed = self.speed + self.accel*TIME_STEP
        self.pos = self.pos + self.speed*TIME_STEP

        # Checking box collisions
        if self.pos[0] - self.radius < 0:
            self.speed[0] = abs(self.speed[0])  # Invert x-speed to bounce off the left wall
        if self.pos[0] + self.radius > BOX_SIZE[0]:
            self.speed[0] = -abs(self.speed[0])  # Invert x-speed to bounce off the right wall

        if self.pos[1] - self.radius < 0:
            self.speed[1] = abs(self.speed[1])  # Invert y-speed to bounce off the top wall
        if self.pos[1] + self.radius > BOX_SIZE[1]:
            self.speed[1] = -abs(self.speed[1])  # Invert y-speed to bounce off the bottom wall

    def calculate_new_velocity(self, other_atom):
        v_relative = self.speed - other_atom.speed
        x_relative = self.pos - other_atom.pos
        x_relative_magnitude_sq = np.dot(x_relative, x_relative)
        
        factor = (2 * other_atom.mass) / (self.mass + other_atom.mass) * np.dot(v_relative, x_relative) / x_relative_magnitude_sq
        new_velocity = self.speed - factor * x_relative
        
        return new_velocity
    
    def collide(self, other_atom):
        if self == other_atom:
            return
        
        new_self_velocity = self.calculate_new_velocity(other_atom)
        self.speed = new_self_velocity
    
        new_other_velocity = other_atom.calculate_new_velocity(self)
        other_atom.speed = new_other_velocity

        # Adjust positions to avoid overlap
        overlap = self.radius + other_atom.radius - np.linalg.norm(self.pos - other_atom.pos)
        if overlap > 0:
            direction = (self.pos - other_atom.pos) / np.linalg.norm(self.pos - other_atom.pos)
            self.pos += 0.5 * overlap * direction
            other_atom.pos -= 0.5 * overlap * direction

### Instanciando as partículas

In [1034]:
particles = {}

min_distance = 10

def calculate_max_particles_y(radius):
    return (BOX_SIZE[1] - 2 * radius) // (2 * radius + min_distance)

for h in range(N_H):
    max_particles_x = (BOX_SIZE[0] - 2 * H_INFOS['radius']) // (2 * H_INFOS['radius'] + min_distance)
    max_particles_y = calculate_max_particles_y(H_INFOS['radius'])

    column = h // max_particles_y
    row = h % max_particles_y

    x_position = BOX_SIZE[0] * 0.05 + column * (2 * H_INFOS['radius'] + min_distance)
    y_position = (BOX_SIZE[1] / (max_particles_y + 1)) * (row + 1)
    
    particles[f'H{h+1}'] = Atom((x_position, y_position), H_INFOS['radius'], H_INFOS['mass'], 'Hydrogen')

for o in range(N_O):
    max_particles_x = (BOX_SIZE[0] - 2 * O_INFOS['radius']) // (2 * O_INFOS['radius'] + min_distance)
    max_particles_y = calculate_max_particles_y(O_INFOS['radius'])

    column = o // max_particles_y
    row = o % max_particles_y

    x_position = BOX_SIZE[0] - BOX_SIZE[0] * 0.05 - column * (2 * O_INFOS['radius'] + min_distance)
    y_position = (BOX_SIZE[1] / (max_particles_y + 1)) * (row + 1)

    particles[f'O{o+1}'] = Atom((x_position, y_position), O_INFOS['radius'], O_INFOS['mass'], 'Oxygen')


## Iniciando a simulação no PyGame

In [1035]:
# Initialize Pygame
pygame.init()

# Create screen
screen = pygame.display.set_mode(BOX_SIZE)
pygame.display.set_caption("Particles Simulation")

# Main loop
running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    # Clear the screen
    screen.fill((50, 50, 50))
    
    # Move and draw particles
    for particle_name, particle in particles.items():
        particle.move()
        color = (255, 255, 255) if particle.type == 'Hydrogen' else (255, 0, 0)
        pygame.draw.circle(screen, color, particle.pos.astype(int), particle.radius)
    
    # Check for collisions and apply them
    for particle_name, particle in particles.items():
        for other_particle_name, other_particle in particles.items():
            if particle_name != other_particle_name:
                if np.linalg.norm(particle.pos - other_particle.pos) <= (particle.radius + other_particle.radius):
                    particle.collide(other_particle)

    # Update the display
    pygame.display.flip()

    # Cap the frame rate
    clock.tick(120)

# Clean up
pygame.quit()